# Clean Markowitz / Risk-Return Notebook

This notebook builds:
- `returns_wide`: cumulative returns per ticker per horizon
- `stddev_wide`: std dev of daily returns per ticker per horizon
- `sharpe_wide`: annualized Sharpe ratio per ticker per horizon

Then it plots:
- Risk/return scatter (colored by horizon)
- Sharpe ratio grouped bar chart (colored by horizon)

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

from returns import (
    get_returns_table,
    get_stddev_table,
    get_sharpe_table,
)

In [2]:
# Load the price panel (generated by a) data_pull.ipynb)
df = pd.read_parquet("ETFs_data.parquet.gzip")
df.rename_axis('date', inplace=True)
df.head()

,Price,High,Low,Open,Volume,Stock_name
date,,,,,,
2018-03-01,8.656540,8.673716,8.656540,8.673716,34800,BXF.TO
2018-03-02,8.682303,8.682303,8.656540,8.656540,4500,BXF.TO
2018-03-05,8.656540,8.673716,8.656540,8.673716,4200,BXF.TO
2018-03-06,8.673715,8.673715,8.673715,8.673715,4900,BXF.TO
2018-03-07,8.647954,8.647954,8.647954,8.647954,4400,BXF.TO


In [3]:
all_tickers = sorted(df['Stock_name'].unique())
all_tickers[:10]

['BXF.TO',
 'CBD.TO',
 'CBH.TO',
 'CBN.TO',
 'CBO.TO',
 'CDZ.TO',
 'CEW.TO',
 'CGL.TO',
 'CGR.TO',
 'CHB.TO']

In [4]:
# Pick a small subset to start (edit freely)
patterns = ['VFV', 'XUU', 'VOO', 'BRK', 'QQ', 'XEF']
subset_tickers = sorted({t for t in all_tickers if any(p in t for p in patterns)})
subset_tickers

['QQC.TO', 'VFV.TO', 'XEF.TO', 'XQQ.TO', 'ZQQ.TO']

In [5]:
subset_df = df[df['Stock_name'].isin(subset_tickers)]
subset_df.head()

,Price,High,Low,Open,Volume,Stock_name
date,,,,,,
2021-05-28,19.342331,19.449462,19.342331,19.449462,900,QQC.TO
2021-05-31,19.303370,19.361805,19.303370,19.361805,400,QQC.TO
2021-06-01,19.215723,19.391032,19.167027,19.391032,1000,QQC.TO
2021-06-02,19.264420,19.264420,19.264420,19.264420,600,QQC.TO
2021-06-03,19.098850,19.118328,19.098850,19.118328,300,QQC.TO


In [6]:
horizons = ['6M', '1Y', '3Y', '5Y']

returns_wide = get_returns_table(subset_df, subset_tickers, horizons)
stddev_wide  = get_stddev_table(subset_df, subset_tickers, horizons)

# Set risk_free_rate_annual to what you want (e.g. 0.03 for 3%)
sharpe_wide  = get_sharpe_table(subset_df, subset_tickers, horizons, risk_free_rate_annual=0.0)

returns_wide

,6M,1Y,3Y,5Y
Ticker,,,,
QQC.TO,-0.026064,0.116402,0.999815,0.846210
VFV.TO,-0.030792,0.117092,0.739733,1.068825
XEF.TO,0.118548,0.159357,0.594541,0.618578
XQQ.TO,0.008821,0.095569,0.812803,0.983927
ZQQ.TO,0.008334,0.095629,0.813875,1.052942


In [7]:
stddev_wide

,6M,1Y,3Y,5Y
Ticker,,,,
QQC.TO,0.018812,0.015828,0.013289,0.013775
VFV.TO,0.015301,0.012232,0.009679,0.009624
XEF.TO,0.011647,0.009779,0.008269,0.008438
XQQ.TO,0.018680,0.015891,0.014347,0.014955
ZQQ.TO,0.018795,0.015997,0.014444,0.014977


In [8]:
sharpe_wide

,6M,1Y,3Y,5Y
Ticker,,,,
QQC.TO,-0.172284,0.463266,1.231909,0.596614
VFV.TO,-0.249642,0.603002,1.319309,1.024360
XEF.TO,1.358334,1.026514,1.281898,0.754762
XQQ.TO,0.059756,0.378855,0.962950,0.618540
ZQQ.TO,0.056101,0.376580,0.957569,0.650740


## Plot: Risk/Return scatter (Plotly)
Each horizon is a different color.

In [9]:
def plot_risk_return_scatter_by_horizon(returns_wide, stddev_wide, horizons, title=None, width=900, height=600):
    horizons = list(horizons)

    ret_long = returns_wide[horizons].reset_index()
    ret_long = ret_long.rename(columns={ret_long.columns[0]: 'ticker'}).melt(
        id_vars='ticker', var_name='horizon', value_name='return'
    )

    vol_long = stddev_wide[horizons].reset_index()
    vol_long = vol_long.rename(columns={vol_long.columns[0]: 'ticker'}).melt(
        id_vars='ticker', var_name='horizon', value_name='stddev'
    )

    plot_df = ret_long.merge(vol_long, on=['ticker', 'horizon']).dropna()

    fig = px.scatter(
        plot_df,
        x='stddev',
        y='return',
        color='horizon',
        text='ticker',
        title=title or 'Risk/Return Scatter by Horizon',
    )
    fig.update_traces(textposition='top center', marker=dict(size=10, opacity=0.85))
    fig.update_layout(
        width=width,
        height=height,
        xaxis_title='Std Dev of Daily Returns',
        yaxis_title='Return',
        legend_title='Horizon',
    )
    return fig

fig = plot_risk_return_scatter_by_horizon(returns_wide, stddev_wide, horizons, width=1000, height=650)
fig.show()

## Plot: Sharpe ratio grouped bar chart (Plotly)

In [10]:
plot_df = (
    sharpe_wide
    .reset_index()
    .rename(columns={sharpe_wide.reset_index().columns[0]: 'ticker'})
    .melt(id_vars='ticker', var_name='horizon', value_name='sharpe')
    .dropna()
)

fig = px.bar(
    plot_df,
    x='ticker',
    y='sharpe',
    color='horizon',
    barmode='group',
    title='Sharpe Ratio by Ticker and Horizon',
)
fig.update_layout(
    width=1100,
    height=550,
    xaxis_title='Ticker',
    yaxis_title='Sharpe Ratio',
    legend_title='Horizon',
)
fig.show()